# Object Detection using TensorFlow

In [ ]:
#import Tensorflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
def cnn_model_fn(features, labels, mode): 
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    #Convolutional Layer 1
    conv1 = tf.layers.conv2d(inputs = input_layer,filters = 32,kernel_size = [5,5],padding = "same",activation = tf.nn.relu)
    
    # Pooling Layer 1
    pooling = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2,2], strides = 2)
    
    # Convolutional Layer 2
    conv2 = tf.layers.conv2d(inputs = pooling,filters = 64, kernel_size = [5,5],padding = "same", activation = tf.nn.relu)
    
    # Pooling Layer 2
    pooling2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2,2],strides = 2)
    
    # Dense Layer
    pool2_flat = tf.reshape(pooling2, [-1,7 * 7 *64])
    dense = tf.layers.dense(inputs = pool2_flat, units = 1024,activation = tf.nn.relu)
    dropout = tf.layers.dropout(inputs = dense, rate = 0.4, training = mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs = dropout, units = 10)
    
    
    # Predictions
    predictions ={
        # Generate predictions for PREDICT & EVAL Modes
        "classes":tf.argmax(input = logits, axis = 1),
        "probabilities":tf.nn.softmax(logits, name = "softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode,predictions = predictions)
    
    # Calculate Loss (For both TRAIN and EVAL Modes)
    onehot_labels = tf.one_hot(indices = tf.cast(labels,tf.int32), depth = 10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels = onehot_labels, logits = logits)
    
    
    # Configure the Training Opperation
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(loss = loss,global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels = labels,predictions = predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode = mode,loss = loss,eval_metric_ops= eval_metric_ops)


In [ ]:
# Loading Training Dataset From TensorFlow (MNIST)

def main():
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    training_data = mnist.train.images # Returns a numpy array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    mnist_classifier = tf.estimator.Estimator(model_fn = cnn_model_fn, model_dir = "/documents/dev-projects/Computer-Vision-Practice")
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors = tensors_to_log, every_n_iter = 50)
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"x": training_data}, y = train_labels, batch_size = 100, num_epochs = None, shuffle = True)
    mnist_classifier.train(input_fn = train_input_fn, steps = 20000, hooks = [logging_hook])
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"x": eval_data}, y = eval_labels, num_epochs =1, shuffle = False)
    eval_results = mnist_classifier.evaluate(input_fn = eval_input_fn)
    print(eval_results)

In [ ]:
# Create Estimator

mnist_classifier = tf.estimator.Estimator(model_fn = cnn_model_fn, model_dir = "/documents/dev-projects/Computer-Vision-Practice")

In [ ]:
# Set up logging for predictions

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors = tensors_to_log, every_n_iter = 50)

In [ ]:
# Train the model

train_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"x": training_data}, y = train_labels, batch_size = 100, num_epochs = None, shuffle = True)
mnist_classifier.train(input_fn = train_input_fn, steps = 20000, hooks = [logging_hook])

In [ ]:
main()